# Tuning Neural Networks with Normalization - Lab 

## Introduction

In this lab you'll build a neural network to perform a regression task.

It is worth noting that getting regression to work with neural networks can be comparatively difficult because the output is unbounded ($\hat y$ can technically range from $-\infty$ to $+\infty$), and the models are especially prone to exploding gradients. This issue makes a regression exercise the perfect learning case for tinkering with normalization and optimization strategies to ensure proper convergence!

## Objectives

In this lab you will: 

- Fit a neural network to normalized data 
- Implement and observe the impact of various initialization techniques 
- Implement and observe the impact of various optimization techniques 

## Load the data 

First, run the following cell to import all the neccessary libraries and classes you will need in this lab. 

In [1]:
# Necessary libraries and classes
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras import initializers
from keras import layers
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from keras import optimizers
from sklearn.model_selection import train_test_split

import warnings
warnings.filterwarnings('ignore')

In this lab, you'll be working with the housing prices data you saw in an earlier section. However, we did a lot of preprocessing for you so you can focus on normalizing numeric features and building neural network models! The following preprocessing steps were taken (all the code can be found in the `data_preprocessing.ipynb` notebook in this repository): 

- The data was split into the training, validate, and test sets 
- All the missing values in numeric columns were replaced by the median of those columns 
- All the missing values in catetgorical columns were replaced with the word 'missing' 
- All the categorical columns were one-hot encoded 

Run the following cells to import the train, validate, and test sets:  

In [2]:
# Load all numeric features
X_train_numeric = pd.read_csv('data/X_train_numeric.csv')
X_val_numeric = pd.read_csv('data/X_val_numeric.csv')
X_test_numeric = pd.read_csv('data/X_test_numeric.csv')

# Load all categorical features
X_train_cat = pd.read_csv('data/X_train_cat.csv')
X_val_cat = pd.read_csv('data/X_val_cat.csv')
X_test_cat = pd.read_csv('data/X_test_cat.csv')

# Load all targets
y_train = pd.read_csv('data/y_train.csv')
y_val = pd.read_csv('data/y_val.csv')
y_test = pd.read_csv('data/y_test.csv')

In [3]:
# Combine all features
X_train = pd.concat([X_train_numeric, X_train_cat], axis=1)
X_val = pd.concat([X_val_numeric, X_val_cat], axis=1)
X_test = pd.concat([X_test_numeric, X_test_cat], axis=1)

# Number of features
n_features = X_train.shape[1]

As a refresher, preview the training data: 

In [4]:
# Preview the data
X_train.head()

,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,...,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_Abnorml,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial
0,80.0,69.0,21453.0,6.0,5.0,1969.0,1969.0,0.0,938.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
1,60.0,79.0,12420.0,7.0,5.0,2001.0,2001.0,0.0,666.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
2,20.0,75.0,9742.0,8.0,5.0,2002.0,2002.0,281.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
3,120.0,39.0,5389.0,8.0,5.0,1995.0,1996.0,0.0,1180.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
4,60.0,85.0,11003.0,10.0,5.0,2008.0,2008.0,160.0,765.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0


## Build a Baseline Model

Building a naive baseline model to compare performance against is a helpful reference point. From there, you can then observe the impact of various tunning procedures which will iteratively improve your model. So, let's do just that! 

In the cell below: 

- Add an input layer with `n_features` units 
- Add two hidden layers, one with 100 and the other with 50 units (make sure you use the `'relu'` activation function) 
- Add an output layer with 1 unit and `'linear'` activation 
- Compile and fit the model 

In [5]:
np.random.seed(123)
baseline_model = Sequential()

# Hidden layer with 100 units
baseline_model.add(layers.Dense(100, activation='relu', 
                                input_shape=(n_features,)))

# Hidden layer with 50 units
baseline_model.add(layers.Dense(50, activation='relu'))

# Output layer
baseline_model.add(layers.Dense(1, activation='linear'))

# Compile the model
baseline_model.compile(optimizer='SGD', 
                       loss='mse', 
                       metrics=['mse'])

# Train the model
baseline_model.fit(X_train, 
                   y_train, 
                   batch_size=32, 
                   epochs=150, 
                   validation_data=(X_val, y_val))

Epoch 1/150
33/33 [==============================] - 0s 13ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 2/150
33/33 [==============================] - 0s 3ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 3/150
33/33 [==============================] - 0s 3ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 4/150
33/33 [==============================] - 0s 3ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 5/150
33/33 [==============================] - 0s 3ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 6/150
33/33 [==============================] - 0s 3ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 7/150
33/33 [==============================] - 0s 3ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 8/150
33/33 [==============================] - 0s 3ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 9/150
33/33 [============================

33/33 [==============================] - 0s 3ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 70/150
33/33 [==============================] - 0s 3ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 71/150
33/33 [==============================] - 0s 3ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 72/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 73/150
33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 74/150
33/33 [==============================] - 0s 3ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 75/150
33/33 [==============================] - 0s 3ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 76/150
33/33 [==============================] - 0s 3ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 77/150
33/33 [==============================] -

33/33 [==============================] - 0s 3ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 138/150
33/33 [==============================] - 0s 3ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 139/150
33/33 [==============================] - 0s 3ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 140/150
33/33 [==============================] - 0s 3ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 141/150
33/33 [==============================] - 0s 3ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 142/150
33/33 [==============================] - 0s 3ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 143/150
33/33 [==============================] - 0s 3ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 144/150
33/33 [==============================] - 0s 3ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 145/150
33/33 [=========================

> _**Notice this extremely problematic behavior: all the values for training and validation loss are "nan". This indicates that the algorithm did not converge. The first solution to this is to normalize the input. From there, if convergence is not achieved, normalizing the output may also be required.**_ 

## Normalize the Input Data 

It's now time to normalize the input data. In the cell below: 

- Assign the column names of all numeric columns to `numeric_columns` 
- Instantiate a `StandardScaler` 
- Fit and transform `X_train_numeric`. Make sure you convert the result into a DataFrame (use `numeric_columns` as the column names) 
- Transform validate and test sets (`X_val_numeric` and `X_test_numeric`), and convert these results into DataFrames as well 
- Use the provided to combine the scaled numerical and categorical features 

In [6]:
# Numeric column names
numeric_columns = X_train_numeric.columns  

# Instantiate StandardScaler
ss_X = StandardScaler()

# Fit and transform train data
X_train_scaled = pd.DataFrame(ss_X.fit_transform(X_train_numeric), columns=numeric_columns)

# Transform validate and test data
X_val_scaled = pd.DataFrame(ss_X.transform(X_val_numeric), columns=numeric_columns)
X_test_scaled = pd.DataFrame(ss_X.transform(X_test_numeric), columns=numeric_columns)

# Combine the scaled numerical features and categorical features
X_train = pd.concat([X_train_scaled, X_train_cat], axis=1)
X_val = pd.concat([X_val_scaled, X_val_cat], axis=1)
X_test = pd.concat([X_test_scaled, X_test_cat], axis=1)

Now run the following cell to compile a neural network model (with the same architecture as before): 

In [7]:
# Model with all normalized inputs
np.random.seed(123)
normalized_input_model = Sequential()
normalized_input_model.add(layers.Dense(100, activation='relu', input_shape=(n_features,)))
normalized_input_model.add(layers.Dense(50, activation='relu'))
normalized_input_model.add(layers.Dense(1, activation='linear'))

# Compile the model
normalized_input_model.compile(optimizer='SGD', 
                               loss='mse', 
                               metrics=['mse'])

In the cell below: 
- Train the `normalized_input_model` on normalized input (`X_train`) and output (`y_train`) 
- Set a batch size of 32 and train for 150 epochs 
- Specify the `validation_data` argument as `(X_val, y_val)` 

In [8]:
# Train the model 
normalized_input_model.fit(X_train,  
                           y_train, 
                           batch_size=32, 
                           epochs=150, 
                           validation_data=(X_val, y_val))

Epoch 1/150
33/33 [==============================] - 0s 10ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 2/150
33/33 [==============================] - 0s 3ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 3/150
33/33 [==============================] - 0s 3ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 4/150
33/33 [==============================] - 0s 3ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 5/150
33/33 [==============================] - 0s 3ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 6/150
33/33 [==============================] - 0s 3ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 7/150
33/33 [==============================] - 0s 3ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 8/150
33/33 [==============================] - 0s 3ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 9/150
33/33 [============================

33/33 [==============================] - 0s 3ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 70/150
33/33 [==============================] - 0s 3ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 71/150
33/33 [==============================] - 0s 3ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 72/150
33/33 [==============================] - 0s 3ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 73/150
33/33 [==============================] - 0s 3ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 74/150
33/33 [==============================] - 0s 3ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 75/150
33/33 [==============================] - 0s 3ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 76/150
33/33 [==============================] - 0s 3ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 77/150
33/33 [==============================] -

33/33 [==============================] - 0s 2ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 138/150
33/33 [==============================] - 0s 3ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 139/150
33/33 [==============================] - 0s 3ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 140/150
33/33 [==============================] - 0s 3ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 141/150
33/33 [==============================] - 0s 3ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 142/150
33/33 [==============================] - 0s 3ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 143/150
33/33 [==============================] - 0s 3ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 144/150
33/33 [==============================] - 0s 3ms/step - loss: nan - mse: nan - val_loss: nan - val_mse: nan
Epoch 145/150
33/33 [=========================

> _**Note that you still haven't achieved convergence! From here, it's time to normalize the output data.**_

## Normalizing the output

Again, use `StandardScaler()` to: 

- Fit and transform `y_train` 
- Transform `y_val` and `y_test` 

In [9]:
# Instantiate StandardScaler
ss_y = StandardScaler()

# Fit and transform train labels
y_train_scaled = ss_y.fit_transform(y_train)

# Transform validate and test labels
y_val_scaled = ss_y.transform(y_val)
y_test_scaled = ss_y.transform(y_test)

In the cell below: 
- Train the `normalized_model` on normalized input (`X_train`) and output (`y_train_scaled`) 
- Set a batch size of 32 and train for 150 epochs 
- Specify the `validation_data` as `(X_val, y_val_scaled)` 

In [10]:
# Model with all normalized inputs and outputs
np.random.seed(123)
normalized_model = Sequential()
normalized_model.add(layers.Dense(100, activation='relu', input_shape=(n_features,)))
normalized_model.add(layers.Dense(50, activation='relu'))
normalized_model.add(layers.Dense(1, activation='linear'))

# Compile the model
normalized_model.compile(optimizer='SGD', 
                         loss='mse', 
                         metrics=['mse']) 

# Train the model
normalized_model.fit(X_train, 
                     y_train_scaled, 
                     batch_size=32, 
                     epochs=150, 
                     validation_data=(X_val, y_val_scaled))

Epoch 1/150
33/33 [==============================] - 0s 10ms/step - loss: 0.4563 - mse: 0.4563 - val_loss: 0.2276 - val_mse: 0.2276
Epoch 2/150
33/33 [==============================] - 0s 3ms/step - loss: 0.2431 - mse: 0.2431 - val_loss: 0.1875 - val_mse: 0.1875
Epoch 3/150
33/33 [==============================] - 0s 3ms/step - loss: 0.1961 - mse: 0.1961 - val_loss: 0.1601 - val_mse: 0.1601
Epoch 4/150
33/33 [==============================] - 0s 3ms/step - loss: 0.1802 - mse: 0.1802 - val_loss: 0.1535 - val_mse: 0.1535
Epoch 5/150
33/33 [==============================] - 0s 3ms/step - loss: 0.1588 - mse: 0.1588 - val_loss: 0.1387 - val_mse: 0.1387
Epoch 6/150
33/33 [==============================] - 0s 3ms/step - loss: 0.1402 - mse: 0.1402 - val_loss: 0.1361 - val_mse: 0.1361
Epoch 7/150
33/33 [==============================] - 0s 3ms/step - loss: 0.1352 - mse: 0.1352 - val_loss: 0.1310 - val_mse: 0.1310
Epoch 8/150
33/33 [==============================] - 0s 3ms/step - loss: 0.1248 - 

33/33 [==============================] - 0s 3ms/step - loss: 0.0217 - mse: 0.0217 - val_loss: 0.1404 - val_mse: 0.1404
Epoch 64/150
33/33 [==============================] - 0s 3ms/step - loss: 0.0212 - mse: 0.0212 - val_loss: 0.1414 - val_mse: 0.1414
Epoch 65/150
33/33 [==============================] - 0s 3ms/step - loss: 0.0215 - mse: 0.0215 - val_loss: 0.1398 - val_mse: 0.1398
Epoch 66/150
33/33 [==============================] - 0s 3ms/step - loss: 0.0210 - mse: 0.0210 - val_loss: 0.1395 - val_mse: 0.1395
Epoch 67/150
33/33 [==============================] - 0s 3ms/step - loss: 0.0201 - mse: 0.0201 - val_loss: 0.1407 - val_mse: 0.1407
Epoch 68/150
33/33 [==============================] - 0s 3ms/step - loss: 0.0204 - mse: 0.0204 - val_loss: 0.1405 - val_mse: 0.1405
Epoch 69/150
33/33 [==============================] - 0s 3ms/step - loss: 0.0204 - mse: 0.0204 - val_loss: 0.1399 - val_mse: 0.1399
Epoch 70/150
33/33 [==============================] - 0s 3ms/step - loss: 0.0194 - mse: 0

Epoch 125/150
33/33 [==============================] - 0s 3ms/step - loss: 0.0097 - mse: 0.0097 - val_loss: 0.1437 - val_mse: 0.1437
Epoch 126/150
33/33 [==============================] - 0s 3ms/step - loss: 0.0097 - mse: 0.0097 - val_loss: 0.1432 - val_mse: 0.1432
Epoch 127/150
33/33 [==============================] - 0s 3ms/step - loss: 0.0095 - mse: 0.0095 - val_loss: 0.1429 - val_mse: 0.1429
Epoch 128/150
33/33 [==============================] - 0s 3ms/step - loss: 0.0092 - mse: 0.0092 - val_loss: 0.1430 - val_mse: 0.1430
Epoch 129/150
33/33 [==============================] - 0s 3ms/step - loss: 0.0090 - mse: 0.0090 - val_loss: 0.1436 - val_mse: 0.1436
Epoch 130/150
33/33 [==============================] - 0s 3ms/step - loss: 0.0091 - mse: 0.0091 - val_loss: 0.1437 - val_mse: 0.1437
Epoch 131/150
33/33 [==============================] - 0s 3ms/step - loss: 0.0090 - mse: 0.0090 - val_loss: 0.1439 - val_mse: 0.1439
Epoch 132/150
33/33 [==============================] - 0s 3ms/step - 

Nicely done! After normalizing both the input and output, the model finally converged. 

- Evaluate the model (`normalized_model`) on training data (`X_train` and `y_train_scaled`) 

In [11]:
# Evaluate the model on training data
normalized_model.evaluate(X_train, y_train_scaled)

33/33 [==============================] - 0s 2ms/step - loss: 0.0089 - mse: 0.0089


[0.00892335083335638, 0.00892335083335638]

- Evaluate the model (`normalized_model`) on validate data (`X_val` and `y_val_scaled`) 

In [12]:
# Evaluate the model on validate data
normalized_model.evaluate(X_val, y_val_scaled)

9/9 [==============================] - 0s 2ms/step - loss: 0.1460 - mse: 0.1460


[0.1460440456867218, 0.1460440456867218]

Since the output is normalized, the metric above is not interpretable. To remedy this: 

- Generate predictions on validate data (`X_val`) 
- Transform these predictions back to original scale using `ss_y` 
- Now you can calculate the RMSE in the original units with `y_val` and `y_val_pred` 

In [13]:
# Generate predictions on validate data
y_val_pred_scaled = normalized_model.predict(X_val)

# Transform the predictions back to original scale
y_val_pred = ss_y.inverse_transform(y_val_pred_scaled)

# RMSE of validate data
np.sqrt(mean_squared_error(y_val, y_val_pred))

30031.250514789357

Great! Now that you have a converged model, you can also experiment with alternative optimizers and initialization strategies to see if you can find a better global minimum. (After all, the current models may have converged to a local minimum.) 

## Using Weight Initializers

In this section you will to use alternative initialization and optimization strategies. At the end, you'll then be asked to select the model which you believe performs the best.  

##  He Initialization

In the cell below, sepcify the following in the first hidden layer:  
  - 100 units 
  - `'relu'` activation 
  - `input_shape` 
  - `kernel_initializer='he_normal'`  

In [14]:
np.random.seed(123)
he_model = Sequential()

# Add the first hidden layer
he_model.add(layers.Dense(100, kernel_initializer='he_normal', activation='relu', input_shape=(n_features,)))

# Add another hidden layer
he_model.add(layers.Dense(50, activation='relu'))

# Add an output layer
he_model.add(layers.Dense(1, activation='linear'))

# Compile the model
he_model.compile(optimizer='SGD', 
                 loss='mse', 
                 metrics=['mse'])

# Train the model
he_model.fit(X_train, 
             y_train_scaled, 
             batch_size=32, 
             epochs=150, 
             validation_data=(X_val, y_val_scaled))

Epoch 1/150
33/33 [==============================] - 0s 12ms/step - loss: 0.4721 - mse: 0.4721 - val_loss: 0.2330 - val_mse: 0.2330
Epoch 2/150
33/33 [==============================] - 0s 3ms/step - loss: 0.2654 - mse: 0.2654 - val_loss: 0.1592 - val_mse: 0.1592
Epoch 3/150
33/33 [==============================] - 0s 3ms/step - loss: 0.1993 - mse: 0.1993 - val_loss: 0.1342 - val_mse: 0.1342
Epoch 4/150
33/33 [==============================] - 0s 4ms/step - loss: 0.1609 - mse: 0.1609 - val_loss: 0.1154 - val_mse: 0.1154
Epoch 5/150
33/33 [==============================] - 0s 3ms/step - loss: 0.1434 - mse: 0.1434 - val_loss: 0.1066 - val_mse: 0.1066
Epoch 6/150
33/33 [==============================] - 0s 3ms/step - loss: 0.1304 - mse: 0.1304 - val_loss: 0.1119 - val_mse: 0.1119
Epoch 7/150
33/33 [==============================] - 0s 3ms/step - loss: 0.1189 - mse: 0.1189 - val_loss: 0.0993 - val_mse: 0.0993
Epoch 8/150
33/33 [==============================] - 0s 3ms/step - loss: 0.1130 - 

33/33 [==============================] - 0s 3ms/step - loss: 0.0248 - mse: 0.0248 - val_loss: 0.0906 - val_mse: 0.0906
Epoch 64/150
33/33 [==============================] - 0s 3ms/step - loss: 0.0241 - mse: 0.0241 - val_loss: 0.0888 - val_mse: 0.0888
Epoch 65/150
33/33 [==============================] - 0s 3ms/step - loss: 0.0238 - mse: 0.0238 - val_loss: 0.0887 - val_mse: 0.0887
Epoch 66/150
33/33 [==============================] - 0s 3ms/step - loss: 0.0236 - mse: 0.0236 - val_loss: 0.0891 - val_mse: 0.0891
Epoch 67/150
33/33 [==============================] - 0s 3ms/step - loss: 0.0231 - mse: 0.0231 - val_loss: 0.0899 - val_mse: 0.0899
Epoch 68/150
33/33 [==============================] - 0s 4ms/step - loss: 0.0230 - mse: 0.0230 - val_loss: 0.0889 - val_mse: 0.0889
Epoch 69/150
33/33 [==============================] - 0s 3ms/step - loss: 0.0224 - mse: 0.0224 - val_loss: 0.0885 - val_mse: 0.0885
Epoch 70/150
33/33 [==============================] - 0s 3ms/step - loss: 0.0221 - mse: 0

Epoch 125/150
33/33 [==============================] - 0s 3ms/step - loss: 0.0116 - mse: 0.0116 - val_loss: 0.0904 - val_mse: 0.0904
Epoch 126/150
33/33 [==============================] - 0s 3ms/step - loss: 0.0115 - mse: 0.0115 - val_loss: 0.0908 - val_mse: 0.0908
Epoch 127/150
33/33 [==============================] - 0s 3ms/step - loss: 0.0112 - mse: 0.0112 - val_loss: 0.0898 - val_mse: 0.0898
Epoch 128/150
33/33 [==============================] - 0s 3ms/step - loss: 0.0114 - mse: 0.0114 - val_loss: 0.0911 - val_mse: 0.0911
Epoch 129/150
33/33 [==============================] - 0s 3ms/step - loss: 0.0111 - mse: 0.0111 - val_loss: 0.0915 - val_mse: 0.0915
Epoch 130/150
33/33 [==============================] - 0s 4ms/step - loss: 0.0110 - mse: 0.0110 - val_loss: 0.0911 - val_mse: 0.0911
Epoch 131/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0109 - mse: 0.0109 - val_loss: 0.0915 - val_mse: 0.0915
Epoch 132/150
33/33 [==============================] - 0s 3ms/step - 

Evaluate the model (`he_model`) on training data (`X_train` and `y_train_scaled`) 

In [15]:
# Evaluate the model on training data
he_model.evaluate(X_train, y_train_scaled)

33/33 [==============================] - 0s 2ms/step - loss: 0.0090 - mse: 0.0090


[0.008982827886939049, 0.008982827886939049]

Evaluate the model (`he_model`) on validate data (`X_train` and `y_train_scaled`) 

In [16]:
# Evaluate the model on validate data
he_model.evaluate(X_val, y_val_scaled)

9/9 [==============================] - 0s 2ms/step - loss: 0.0918 - mse: 0.0918


[0.09183327853679657, 0.09183327853679657]

## Lecun Initialization 

In the cell below, sepcify the following in the first hidden layer:  
  - 100 units 
  - `'relu'` activation 
  - `input_shape` 
  - `kernel_initializer='lecun_normal'` 

In [17]:
np.random.seed(123)
lecun_model = Sequential()

# Add the first hidden layer
lecun_model.add(layers.Dense(100, kernel_initializer='lecun_normal', activation='relu', input_shape=(n_features,)))

# Add another hidden layer
lecun_model.add(layers.Dense(50, activation='relu'))

# Add an output layer
lecun_model.add(layers.Dense(1, activation='linear'))

# Compile the model
lecun_model.compile(optimizer='SGD', 
                    loss='mse', 
                    metrics=['mse'])

# Train the model
lecun_model.fit(X_train, 
                y_train_scaled, 
                batch_size=32, 
                epochs=150, 
                validation_data=(X_val, y_val_scaled))

Epoch 1/150
33/33 [==============================] - 0s 11ms/step - loss: 0.5763 - mse: 0.5763 - val_loss: 0.2069 - val_mse: 0.2069
Epoch 2/150
33/33 [==============================] - 0s 3ms/step - loss: 0.2622 - mse: 0.2622 - val_loss: 0.1395 - val_mse: 0.1395
Epoch 3/150
33/33 [==============================] - 0s 3ms/step - loss: 0.1998 - mse: 0.1998 - val_loss: 0.1225 - val_mse: 0.1225
Epoch 4/150
33/33 [==============================] - 0s 4ms/step - loss: 0.1700 - mse: 0.1700 - val_loss: 0.1142 - val_mse: 0.1142
Epoch 5/150
33/33 [==============================] - 0s 3ms/step - loss: 0.1615 - mse: 0.1615 - val_loss: 0.1266 - val_mse: 0.1266
Epoch 6/150
33/33 [==============================] - 0s 3ms/step - loss: 0.1431 - mse: 0.1431 - val_loss: 0.1068 - val_mse: 0.1068
Epoch 7/150
33/33 [==============================] - 0s 3ms/step - loss: 0.1302 - mse: 0.1302 - val_loss: 0.1116 - val_mse: 0.1116
Epoch 8/150
33/33 [==============================] - 0s 3ms/step - loss: 0.1208 - 

33/33 [==============================] - 0s 3ms/step - loss: 0.0224 - mse: 0.0224 - val_loss: 0.1116 - val_mse: 0.1116
Epoch 64/150
33/33 [==============================] - 0s 3ms/step - loss: 0.0226 - mse: 0.0226 - val_loss: 0.1110 - val_mse: 0.1110
Epoch 65/150
33/33 [==============================] - 0s 3ms/step - loss: 0.0224 - mse: 0.0224 - val_loss: 0.1114 - val_mse: 0.1114
Epoch 66/150
33/33 [==============================] - 0s 3ms/step - loss: 0.0215 - mse: 0.0215 - val_loss: 0.1107 - val_mse: 0.1107
Epoch 67/150
33/33 [==============================] - 0s 3ms/step - loss: 0.0221 - mse: 0.0221 - val_loss: 0.1100 - val_mse: 0.1100
Epoch 68/150
33/33 [==============================] - 0s 3ms/step - loss: 0.0207 - mse: 0.0207 - val_loss: 0.1102 - val_mse: 0.1102
Epoch 69/150
33/33 [==============================] - 0s 3ms/step - loss: 0.0208 - mse: 0.0208 - val_loss: 0.1102 - val_mse: 0.1102
Epoch 70/150
33/33 [==============================] - 0s 3ms/step - loss: 0.0202 - mse: 0

Epoch 125/150
33/33 [==============================] - 0s 4ms/step - loss: 0.0104 - mse: 0.0104 - val_loss: 0.1140 - val_mse: 0.1140
Epoch 126/150
33/33 [==============================] - 0s 3ms/step - loss: 0.0102 - mse: 0.0102 - val_loss: 0.1159 - val_mse: 0.1159
Epoch 127/150
33/33 [==============================] - 0s 3ms/step - loss: 0.0103 - mse: 0.0103 - val_loss: 0.1138 - val_mse: 0.1138
Epoch 128/150
33/33 [==============================] - 0s 3ms/step - loss: 0.0099 - mse: 0.0099 - val_loss: 0.1130 - val_mse: 0.1130
Epoch 129/150
33/33 [==============================] - 0s 3ms/step - loss: 0.0097 - mse: 0.0097 - val_loss: 0.1158 - val_mse: 0.1158
Epoch 130/150
33/33 [==============================] - 0s 3ms/step - loss: 0.0097 - mse: 0.0097 - val_loss: 0.1126 - val_mse: 0.1126
Epoch 131/150
33/33 [==============================] - 0s 3ms/step - loss: 0.0095 - mse: 0.0095 - val_loss: 0.1138 - val_mse: 0.1138
Epoch 132/150
33/33 [==============================] - 0s 4ms/step - 

Evaluate the model (`lecun_model`) on training data (`X_train` and `y_train_scaled`) 

In [18]:
# Evaluate the model on training data
lecun_model.evaluate(X_train, y_train_scaled)

33/33 [==============================] - 0s 2ms/step - loss: 0.0075 - mse: 0.0075


[0.007512978278100491, 0.007512978278100491]

Evaluate the model (`lecun_model`) on validate data (`X_train` and `y_train_scaled`) 

In [19]:
# Evaluate the model on validate data
lecun_model.evaluate(X_val, y_val_scaled)

9/9 [==============================] - 0s 2ms/step - loss: 0.1147 - mse: 0.1147


[0.11470469832420349, 0.11470469832420349]

Not much of a difference, but a useful note to consider when tuning your network. Next, let's investigate the impact of various optimization algorithms.

## RMSprop 

Compile the `rmsprop_model` with: 

- `'rmsprop'` as the optimizer 
- track `'mse'` as the loss and metric  

In [20]:
np.random.seed(123)
rmsprop_model = Sequential()
rmsprop_model.add(layers.Dense(100, activation='relu', input_shape=(n_features,)))
rmsprop_model.add(layers.Dense(50, activation='relu'))
rmsprop_model.add(layers.Dense(1, activation='linear'))

# Compile the model
rmsprop_model.compile(optimizer='rmsprop', 
                      loss='mse', 
                      metrics=['mse'])

# Train the model
rmsprop_model.fit(X_train, 
                  y_train_scaled, 
                  batch_size=32, 
                  epochs=150, 
                  validation_data=(X_val, y_val_scaled))

Epoch 1/150
33/33 [==============================] - 0s 11ms/step - loss: 0.3365 - mse: 0.3365 - val_loss: 0.1833 - val_mse: 0.1833
Epoch 2/150
33/33 [==============================] - 0s 3ms/step - loss: 0.1691 - mse: 0.1691 - val_loss: 0.1143 - val_mse: 0.1143
Epoch 3/150
33/33 [==============================] - 0s 3ms/step - loss: 0.1268 - mse: 0.1268 - val_loss: 0.1744 - val_mse: 0.1744
Epoch 4/150
33/33 [==============================] - 0s 3ms/step - loss: 0.0957 - mse: 0.0957 - val_loss: 0.1082 - val_mse: 0.1082
Epoch 5/150
33/33 [==============================] - 0s 2ms/step - loss: 0.0743 - mse: 0.0743 - val_loss: 0.1322 - val_mse: 0.1322
Epoch 6/150
33/33 [==============================] - 0s 3ms/step - loss: 0.0627 - mse: 0.0627 - val_loss: 0.1234 - val_mse: 0.1234
Epoch 7/150
33/33 [==============================] - 0s 4ms/step - loss: 0.0500 - mse: 0.0500 - val_loss: 0.1655 - val_mse: 0.1655
Epoch 8/150
33/33 [==============================] - 0s 4ms/step - loss: 0.0466 - 

33/33 [==============================] - 0s 5ms/step - loss: 0.0078 - mse: 0.0078 - val_loss: 0.1228 - val_mse: 0.1228
Epoch 64/150
33/33 [==============================] - 0s 3ms/step - loss: 0.0078 - mse: 0.0078 - val_loss: 0.1205 - val_mse: 0.1205
Epoch 65/150
33/33 [==============================] - 0s 3ms/step - loss: 0.0081 - mse: 0.0081 - val_loss: 0.1236 - val_mse: 0.1236
Epoch 66/150
33/33 [==============================] - 0s 3ms/step - loss: 0.0059 - mse: 0.0059 - val_loss: 0.1110 - val_mse: 0.1110
Epoch 67/150
33/33 [==============================] - 0s 4ms/step - loss: 0.0070 - mse: 0.0070 - val_loss: 0.1063 - val_mse: 0.1063
Epoch 68/150
33/33 [==============================] - 0s 4ms/step - loss: 0.0070 - mse: 0.0070 - val_loss: 0.1234 - val_mse: 0.1234
Epoch 69/150
33/33 [==============================] - 0s 3ms/step - loss: 0.0072 - mse: 0.0072 - val_loss: 0.1204 - val_mse: 0.1204
Epoch 70/150
33/33 [==============================] - 0s 3ms/step - loss: 0.0068 - mse: 0

Epoch 125/150
33/33 [==============================] - 0s 4ms/step - loss: 0.0044 - mse: 0.0044 - val_loss: 0.1003 - val_mse: 0.1003
Epoch 126/150
33/33 [==============================] - 0s 3ms/step - loss: 0.0033 - mse: 0.0033 - val_loss: 0.1003 - val_mse: 0.1003
Epoch 127/150
33/33 [==============================] - 0s 3ms/step - loss: 0.0048 - mse: 0.0048 - val_loss: 0.0966 - val_mse: 0.0966
Epoch 128/150
33/33 [==============================] - 0s 3ms/step - loss: 0.0037 - mse: 0.0037 - val_loss: 0.0940 - val_mse: 0.0940
Epoch 129/150
33/33 [==============================] - 0s 3ms/step - loss: 0.0054 - mse: 0.0054 - val_loss: 0.0983 - val_mse: 0.0983
Epoch 130/150
33/33 [==============================] - 0s 4ms/step - loss: 0.0035 - mse: 0.0035 - val_loss: 0.1011 - val_mse: 0.1011
Epoch 131/150
33/33 [==============================] - 0s 3ms/step - loss: 0.0045 - mse: 0.0045 - val_loss: 0.1024 - val_mse: 0.1024
Epoch 132/150
33/33 [==============================] - 0s 4ms/step - 

Evaluate the model (`rmsprop_model`) on training data (`X_train` and `y_train_scaled`) 

In [21]:
# Evaluate the model on training data
rmsprop_model.evaluate(X_train, y_train_scaled)

33/33 [==============================] - 0s 2ms/step - loss: 0.0023 - mse: 0.0023


[0.0022634509950876236, 0.0022634509950876236]

Evaluate the model (`rmsprop_model`) on training data (`X_train` and `y_train_scaled`) 

In [22]:
# Evaluate the model on validate data
rmsprop_model.evaluate(X_val, y_val_scaled)

9/9 [==============================] - 0s 2ms/step - loss: 0.0919 - mse: 0.0919


[0.09189093112945557, 0.09189093112945557]

## Adam 

Compile the `adam_model` with: 

- `'Adam'` as the optimizer 
- track `'mse'` as the loss and metric  

In [23]:
np.random.seed(123)
adam_model = Sequential()
adam_model.add(layers.Dense(100, activation='relu', input_shape=(n_features,)))
adam_model.add(layers.Dense(50, activation='relu'))
adam_model.add(layers.Dense(1, activation='linear'))

# Compile the model
adam_model.compile(optimizer='Adam', 
                   loss='mse', 
                   metrics=['mse'])

# Train the model
adam_model.fit(X_train, 
               y_train_scaled, 
               batch_size=32, 
               epochs=150, 
               validation_data=(X_val, y_val_scaled))

Epoch 1/150
33/33 [==============================] - 1s 19ms/step - loss: 0.3234 - mse: 0.3234 - val_loss: 0.1120 - val_mse: 0.1120
Epoch 2/150
33/33 [==============================] - 0s 3ms/step - loss: 0.1591 - mse: 0.1591 - val_loss: 0.1130 - val_mse: 0.1130
Epoch 3/150
33/33 [==============================] - 0s 3ms/step - loss: 0.0993 - mse: 0.0993 - val_loss: 0.1048 - val_mse: 0.1048
Epoch 4/150
33/33 [==============================] - 0s 4ms/step - loss: 0.0731 - mse: 0.0731 - val_loss: 0.0981 - val_mse: 0.0981
Epoch 5/150
33/33 [==============================] - 0s 4ms/step - loss: 0.0583 - mse: 0.0583 - val_loss: 0.1062 - val_mse: 0.1062
Epoch 6/150
33/33 [==============================] - 0s 4ms/step - loss: 0.0454 - mse: 0.0454 - val_loss: 0.0990 - val_mse: 0.0990
Epoch 7/150
33/33 [==============================] - 0s 4ms/step - loss: 0.0346 - mse: 0.0346 - val_loss: 0.0977 - val_mse: 0.0977
Epoch 8/150
33/33 [==============================] - 0s 4ms/step - loss: 0.0289 - 

Epoch 63/150
33/33 [==============================] - 0s 5ms/step - loss: 0.0017 - mse: 0.0017 - val_loss: 0.0985 - val_mse: 0.0985
Epoch 64/150
33/33 [==============================] - 0s 3ms/step - loss: 0.0015 - mse: 0.0015 - val_loss: 0.1038 - val_mse: 0.1038
Epoch 65/150
33/33 [==============================] - 0s 4ms/step - loss: 0.0023 - mse: 0.0023 - val_loss: 0.1001 - val_mse: 0.1001
Epoch 66/150
33/33 [==============================] - 0s 3ms/step - loss: 0.0017 - mse: 0.0017 - val_loss: 0.1021 - val_mse: 0.1021
Epoch 67/150
33/33 [==============================] - 0s 3ms/step - loss: 0.0011 - mse: 0.0011 - val_loss: 0.0995 - val_mse: 0.0995
Epoch 68/150
33/33 [==============================] - 0s 4ms/step - loss: 0.0015 - mse: 0.0015 - val_loss: 0.1041 - val_mse: 0.1041
Epoch 69/150
33/33 [==============================] - 0s 3ms/step - loss: 0.0019 - mse: 0.0019 - val_loss: 0.0938 - val_mse: 0.0938
Epoch 70/150
33/33 [==============================] - 0s 3ms/step - loss: 0.

33/33 [==============================] - 0s 3ms/step - loss: 0.0031 - mse: 0.0031 - val_loss: 0.0833 - val_mse: 0.0833
Epoch 125/150
33/33 [==============================] - 0s 3ms/step - loss: 0.0057 - mse: 0.0057 - val_loss: 0.0946 - val_mse: 0.0946
Epoch 126/150
33/33 [==============================] - 0s 4ms/step - loss: 0.0067 - mse: 0.0067 - val_loss: 0.0795 - val_mse: 0.0795
Epoch 127/150
33/33 [==============================] - 0s 3ms/step - loss: 0.0059 - mse: 0.0059 - val_loss: 0.0903 - val_mse: 0.0903
Epoch 128/150
33/33 [==============================] - 0s 4ms/step - loss: 0.0047 - mse: 0.0047 - val_loss: 0.0832 - val_mse: 0.0832
Epoch 129/150
33/33 [==============================] - 0s 3ms/step - loss: 0.0050 - mse: 0.0050 - val_loss: 0.0872 - val_mse: 0.0872
Epoch 130/150
33/33 [==============================] - 0s 4ms/step - loss: 0.0050 - mse: 0.0050 - val_loss: 0.0858 - val_mse: 0.0858
Epoch 131/150
33/33 [==============================] - 0s 3ms/step - loss: 0.0061 -

Evaluate the model (`adam_model`) on training data (`X_train` and `y_train_scaled`) 

In [24]:
# Evaluate the model on training data
adam_model.evaluate(X_train, y_train_scaled)

33/33 [==============================] - 0s 2ms/step - loss: 0.0018 - mse: 0.0018


[0.0018165302462875843, 0.0018165302462875843]

Evaluate the model (`adam_model`) on training data (`X_train` and `y_train_scaled`) 

In [25]:
# Evaluate the model on validate data
adam_model.evaluate(X_val, y_val_scaled)

9/9 [==============================] - 0s 2ms/step - loss: 0.0776 - mse: 0.0776


[0.07763388007879257, 0.07763388007879257]

## Select a Final Model

Now, select the model with the best performance based on the training and validation sets. Evaluate this top model using the test set!

In [26]:
# Evaluate the best model on test data
rmsprop_model.evaluate(X_test, y_test_scaled)

5/5 [==============================] - 0s 2ms/step - loss: 0.1271 - mse: 0.1271


[0.1270861029624939, 0.1270861029624939]

As earlier, this metric is hard to interpret because the output is scaled. 

- Generate predictions on test data (`X_test`) 
- Transform these predictions back to original scale using `ss_y` 
- Now you can calculate the RMSE in the original units with `y_test` and `y_test_pred` 

In [27]:
# Generate predictions on test data
y_test_pred_scaled = rmsprop_model.predict(X_test)

# Transform the predictions back to original scale
y_test_pred = ss_y.inverse_transform(y_test_pred_scaled)

# MSE of test data
np.sqrt(mean_squared_error(y_test, y_test_pred))

28014.350258420855

## Summary  

In this lab, you worked to ensure your model converged properly by normalizing both the input and output. Additionally, you also investigated the impact of varying initialization and optimization routines.